# Datos ECH

Los datos de la ECH se importan desde archivos CSV que corresponden a cada año. En primer lugar se carga cada uno de ellos en un dataframe de pandas.

In [2]:
import pandas as pd
import os.path

firstECHyear = 2017
lastECHyear = 2019


for year in range(firstECHyear,lastECHyear+1):
    pickleFile = ".pickles/ECH/H_{year}_Terceros.pkl".format(year=year)
    if not os.path.exists(pickleFile):
        path = "ECH/{year}/H_{year}_Terceros.sav".format(year=year)
        df = pd.read_spss(path)

        # Se reemplaza las secciones y segmentos vacios con '00'  y '000' respectivamente 
        df.loc[df.secc == '', 'secc'] = '00'
        df.loc[df.segm == '', 'segm'] = '000'

        # Se transforma seccion y segmento de int a float
        df['secc'] = df['secc'].astype(float)
        df['segm'] = df['segm'].astype(float)

        # Se almacena esta primera transformacion en un archivo .pkl
        df.to_pickle(pickleFile)
        print("{year} procesado".format(year=year))

# Se cargan todos los datagrames en un diccionario
hogaresDfDict = {}
for year in range(firstECHyear,lastECHyear+1):
    hogaresDfDict[str(year)] = pd.read_pickle(".pickles/ECH/H_{year}_Terceros.pkl".format(year=year))


2017 done
2018 done
2019 done


In [3]:
# Como ejemplo se muestran las primeras cuatro filas de los datos cargados de la ECH para el año 2019
hogaresDfDict['2019'].head()

,numero,anio,mes,dpto,nomdpto,secc,segm,loc_agr_13,nom_loc_agr_13,ccz,...,HT11,HT13,ht14,ht19,YHOG,YSVL,lp_06,li_06,pobre06,indigente06
0,2019000001,2019,Agosto,Rocha,ROCHA,0.0,0.0,35050,Agrupación localidades entre 5.000 y 20.000 ha...,0.0,...,31517.0,6500.0,0.0,2.0,2017.0,25017.0,17097.383456,3328.0,No pobre,No indigente
1,2019000002,2019,Febrero,Salto,SALTO,0.0,0.0,36000,Agrupación localidades menores a 5.000 habitan...,0.0,...,51702.0,7000.0,0.0,2.0,0.0,44702.0,16492.923793,3152.0,No pobre,No indigente
2,2019000003,2019,Noviembre,Montevideo,MONTEVIDEO,11.0,209.0,01010,Montevideo,9.0,...,33986.5,5000.0,0.0,2.0,5018.5,28986.5,26483.555673,3758.0,No pobre,No indigente
3,2019000004,2019,Agosto,Montevideo,MONTEVIDEO,7.0,206.0,01010,Montevideo,2.0,...,72924.0,17000.0,0.0,2.0,2262.0,55924.0,25973.613807,3631.0,No pobre,No indigente
4,2019000005,2019,Noviembre,Canelones,CANELONES,0.0,0.0,03023,"Canelones y otras (Paso Espinosa, Paso Palomeque)",0.0,...,78196.0,15000.0,0.0,4.0,2032.0,63196.0,32230.736251,3459.0,No pobre,No indigente


Ahora se filtra, dentro de los datos de cada año, aquellos que pertenecen al departamento de Montevideo y se genera un csv para cada año.

In [4]:
# Se crea un diccionario de siendo las claves cada año del rango definido con los datos de ECH previamente almecenados y se filtran aquellos asociados al departamento de Montevideo.
ECHMontevideoDfDict = {}
for year in range(firstECHyear,lastECHyear+1):
    ECHMontevideoDfDict[str(year)] = hogaresDfDict[str(year)].where(hogaresDfDict[str(year)]['nomdpto'] == 'MONTEVIDEO').dropna()

# Se almacenan los datos en un archivo .csv para cada año
for year in range(firstECHyear,lastECHyear+1):
    ECHMontevideoDfDict[str(year)].to_csv("ECH/{year}_Montevideo.csv".format(year=year), index=False)
    print(f"{year} procesado")

2017 procesado
2018 procesado
2019 procesado


Una vez obtenidos los datos de los segmentos y secciones unicamente de Montevideo interesa trabajar con aquellos segmentos y secciones que fueron encuestadas todos los años del rango definido.

In [5]:
# En primer lugar se obtienen aquellos segmentos que fueron encuestados todos los años para el rango definido

seccionesConDatos = {}
for year in range(firstECHyear,lastECHyear+1):
    seccionesConDatos[str(year)] = ECHMontevideoDfDict[str(year)][['segm','secc']].drop_duplicates()

dfAcumulador = seccionesConDatos[str(firstECHyear)]

for seccion in seccionesConDatos.values():
    dfAcumulador = dfAcumulador.merge(seccion, on=['segm','secc'], how='inner')

print("{segmentos} segmentos se evaluaron todos los años".format(segmentos = dfAcumulador.shape[0]))

# Se alamacena en un csv
dfAcumulador.to_csv("ECH/SEGMENTOS_TODOS_AÑOS.csv".format(first=firstECHyear, last=lastECHyear), index=False)
    

854 segmentos se evaluaron todos los años


In [6]:
# Se ejecuta un join con el dataframe que contiene todos los datos de la ECH con el dataframe que contiene los segmentos que fueron encuestados todos los años
# la intencion detras de este join es filtrar aquellos pares segmento-seccion que fueron encuestados todos los años

ECHMontevideoDfDictTodosAños = {}
for year, dfECHanio in ECHMontevideoDfDict.items():
    ECHMontevideoDfDictTodosAños[year] = dfECHanio.merge(dfAcumulador, on=['segm','secc'], how='inner')
    

Se cambia el nombre original de las variables para que sean mas mnemotecnicos (coincide con los nombres presentados en el informe). Finalmente se toman solo las variables con las vamos a trabajar.

In [7]:
for year in range(firstECHyear, lastECHyear+1):
    ECHMontevideoDfDictTodosAños[str(year)].rename(
        columns={'HT11': 'ingresos_hogar'}, inplace=True)
    ECHMontevideoDfDictTodosAños[str(year)].rename(
        columns={'ht11': 'ingresos_hogar'}, inplace=True)
    ECHMontevideoDfDictTodosAños[str(year)].rename(
        columns={'estred13': 'nivel_económico_hogar'}, inplace=True)
    ECHMontevideoDfDictTodosAños[str(year)].rename(
        columns={'c1': 'tipo_vivienda'}, inplace=True)
    ECHMontevideoDfDictTodosAños[str(year)].rename(
        columns={'d9': 'cant_habitaciones'}, inplace=True)
    ECHMontevideoDfDictTodosAños[str(year)].rename(
        columns={'d20': 'energia_cocinar'}, inplace=True)
    ECHMontevideoDfDictTodosAños[str(year)].rename(
        columns={'d21_1': 'tiene_calefon'}, inplace=True)
    ECHMontevideoDfDictTodosAños[str(year)].rename(
        columns={'d21_5': 'cant_televisores'}, inplace=True)
    ECHMontevideoDfDictTodosAños[str(year)].rename(
        columns={'d260': 'energia_calefaccionar'}, inplace=True)
    ECHMontevideoDfDictTodosAños[str(year)].rename(
        columns={'d21_14_1': 'cant_ac'}, inplace=True)
    ECHMontevideoDfDictTodosAños[str(year)].rename(
        columns={'c2': 'material_pared'}, inplace=True)
    ECHMontevideoDfDictTodosAños[str(year)].rename(
        columns={'c3': 'material_techo'}, inplace=True)
    ECHMontevideoDfDictTodosAños[str(year)].rename(
        columns={'c4': 'material_piso'}, inplace=True)
    ECHMontevideoDfDictTodosAños[str(year)] = ECHMontevideoDfDictTodosAños[str(year)][[
        'anio', 'mes', 'dpto', 'nomdpto', 'secc', 'segm', 'pesoano', 'ingresos_hogar',
        'nivel_económico_hogar', 'tipo_vivienda', 'cant_habitaciones', 'energia_cocinar',
        'tiene_calefon', 'cant_televisores', 'energia_calefaccionar', 'cant_ac',
        'material_pared', 'material_techo', 'material_piso'
    ]]

In [8]:
ECHMontevideoDfDictTodosAños['2019'].head()

,anio,mes,dpto,nomdpto,secc,segm,pesoano,ingresos_hogar,nivel_económico_hogar,tipo_vivienda,cant_habitaciones,energia_cocinar,tiene_calefon,cant_televisores,energia_calefaccionar,cant_ac,material_pared,material_techo,material_piso
0,2019,Noviembre,Montevideo,MONTEVIDEO,11.0,209.0,27.0,33986.5,Montevideo Medio Bajo,Casa,2.0,Supergás,No,Sí,Energía eléctrica,0.0,"Ladrillos, ticholos o bloques con terminaciones",Liviano sin cielo raso,Solo contrapiso sin piso
1,2019,Noviembre,Montevideo,MONTEVIDEO,11.0,209.0,40.0,18000.0,Montevideo Medio Bajo,Apartamento en edificio de una planta,2.0,Supergás,No,No,Energía eléctrica,0.0,"Ladrillos, ticholos o bloques con terminaciones",Liviano sin cielo raso,Solo contrapiso sin piso
2,2019,Agosto,Montevideo,MONTEVIDEO,11.0,209.0,29.0,44524.0,Montevideo Medio Bajo,Casa,3.0,Supergás,No,No,Energía eléctrica,0.0,"Ladrillos, ticholos o bloques con terminaciones",Planchada de hormigón con protección (tejas u ...,"Cerámica, parqué, baldosas, moqueta o linóleo"
3,2019,Setiembre,Montevideo,MONTEVIDEO,11.0,209.0,25.0,32000.0,Montevideo Medio Bajo,Apartamento en edificio de una planta,2.0,Energía eléctrica,No,No,Energía eléctrica,0.0,"Ladrillos, ticholos o bloques sin terminaciones",Liviano sin cielo raso,Solo contrapiso sin piso
4,2019,Setiembre,Montevideo,MONTEVIDEO,11.0,209.0,45.0,52186.5,Montevideo Medio Bajo,Apartamento en edificio de una planta,4.0,Supergás,Sí,Sí,Energía eléctrica,0.0,Materiales livianos sin revestimiento,Liviano sin cielo raso,"Cerámica, parqué, baldosas, moqueta o linóleo"


## T3

A continuacion se trata con el problema de la diferencia de formatos en las categorias de distintos años de la encuestam, por eso que se debe uniificar un mismo formato para cada variable.
Ademas se ejecuta una transformacion sobre algunas variables categoricas agrupando categorias que permitan hacer un conteo más facilmente, por ejemplo el caso de la variable tipo_vivienda donde agrupamos en dos categorias "casa" y "apartamento" registrandolas como 0 y 1 respectivamente. 

In [9]:
for year in range(firstECHyear, lastECHyear+1):
    ECHMontevideoDfDictTodosAños[str(year)].replace(
        ['Sí', 'No'], [1, 0], inplace=True)
    ECHMontevideoDfDictTodosAños[str(year)].replace({'tipo_vivienda': {
        'Apartamento o casa en complejo habitacional': 0,
        'Apartamento en edificio de altura': 0,
        'Apartamento en edificio de una planta': 1,
        'Local no construido para vivienda': 1,
        'Casa': 1,
    }, 'material_pared': {
        'Ladrillos, ticholos o bloques con terminaciones': 1,
        'Ladrillos, ticholos o bloques, con terminaciones': 1,
        'Ladrillos, ticholos o bloques sin terminaciones': 1,
        'Ladrillos, ticholos o bloques, sin terminaciones': 1,
        'Materiales livianos con revestimiento': 0,
        'Materiales livianos sin revestimiento': 0,
        'Adobe': 0,
        'Materiales de desecho': 0, }
    , 'material_techo': {
        'Planchada de hormigón con protección (tejas u otros)': 1,
        'Planchada de hormigón sin protección': 1,
        'Liviano con cielo raso': 0,
        'Liviano sin cielo raso': 0,
        'Quincha': 0,
        'Materiales de desecho': 0, }
    , 'material_piso': {
        'Cerámica, parqué, baldosas, moqueta o linóleo': 1,
        'Cerámica, parqué, moqueta, linóleo': 1,
        'Solo contrapiso sin piso': 0,
        'Alisado de hormigón': 0,
        'Tierra sin piso ni contrapiso': 0, }
    }, inplace=True)

A continuacion se implementa la transformacion de forma de obtener una misma fila para cada segmento-seccion. Para esto es que, a partir de cada variable se crea una nueva que busca reunir la información de la misma pero de forma generalizada para todo el segmento en que se encuentra.

In [10]:
from hashlib import new
from sklearn.utils.extmath import weighted_mode
import numpy as np


def aggregateECH(dfGroup, args):

    filaResumen = pd.Series(dtype=object)

    # se suma la variable peso año
    filaResumen['pesoano'] = dfGroup['pesoano'].sum()


    # porcentaje_casas: en este caso, se mapea viviendas a casa o apartamento para tener un porcentaje definido.
    # 1: casa, 0: Apartamento
    filaResumen['porcentaje_casas'] = np.average(
        dfGroup['tipo_vivienda'], weights=dfGroup['pesoano'])


    # porcentaje_pared_ladrillo: Se calcula agrupando por seccion-segmento el porcentaje ponderado 
    # de todas las entradas que en la variable material_pared tienen la categorias ladrillo
    filaResumen['porcentaje_pared_ladrillo'] = np.average(
        dfGroup['material_pared'], weights=dfGroup['pesoano'])


    # porcentaje_techo_hormigon: Se calcula agrupando por seccion-segmento el porcentaje ponderado 
    # de todas las entradas que en la variable material_techo tienen la categorias hormigon
    filaResumen['porcentaje_techo_hormigon'] = np.average(
        dfGroup['material_techo'], weights=dfGroup['pesoano'])

    # porcentaje_techo_hormigon: Se calcula agrupando por seccion-segmento el porcentaje ponderado 
    # de todas las entradas que en la variable material_techo tienen la categorias hormigon    
    filaResumen['porcentaje_piso_con_terminacion'] = np.average(
        dfGroup['material_piso'], weights=dfGroup['pesoano'])

    # cant_habitaciones_prom: Se calcula, agrupando por seccion-segmento, el promedio ponderado de la variable cant_habitaciones
    filaResumen['cant_habitaciones_prom'] = np.average(dfGroup['cant_habitaciones'], weights=dfGroup['pesoano'])

    # porcentaje_cocina_electrica: Se calcula agrupando por seccion-segmento el porcentaje de ponderado 
    # de todas las entradas que en la variable energia_cocinar tienen la categorias hormigon
    filaResumen['porcentaje_cocina_electrica'] = round(dfGroup.where(
        dfGroup['energia_cocinar'] == "Energía eléctrica")['pesoano'].sum() / filaResumen['pesoano'], 2)

    # TODO: ver el calculo de esta variable
    # tiene_calefon_prom: Se calcula, agrupando por seccion-segmento, el promedio ponderado de todas las
    # entradas de la variable tiene_calefon 
    filaResumen['tiene_calefon_prom'] = np.average(
        dfGroup['tiene_calefon'].astype('float'), weights=dfGroup['pesoano'])

    # cant_televisores_prom: Se calcula agrupando por seccion-segmento el promedio ponderado de todas las
    # entradas para la variable cant_televisores
    filaResumen['cant_televisores_prom'] = np.average(
        dfGroup['cant_televisores'], weights=dfGroup['pesoano'])

    # ingresos_hogar_prom: Se calcula agrupando por seccion-segmento el promedio ponderado de todas las
    # entradas para la variable ingresos_hogar
    filaResumen['ingresos_hogar_prom'] = np.average(
        dfGroup['ingresos_hogar'], weights=dfGroup['pesoano'])


    # cantidad_ac_prom: Se calcula agrupando por seccion-segmento el promedio ponderado de todas las
    # entradas para la variable cantidad_ac
    if (args == 2013):
        filaResumen['cant_ac'] = np.NaN
    else:
        filaResumen['cant_ac_prom'] = np.average(
            dfGroup['cant_ac'], weights=dfGroup['pesoano'])

    # porcentaje_energia_calefaccioanr: Se calcula agrupando por seccion-segmento el porcentaje ponderado 
    # de todas las entradas que en la variable energia_calefaccionar tienen la categoria electrica
    if (args == 2013):
        filaResumen['energia_calefaccionar'] = np.NaN
    else:
        filaResumen['porcentaje_calefaccio_electrica'] = round(dfGroup.where(
            dfGroup['energia_calefaccionar'] == "Energía eléctrica")['pesoano'].sum() / filaResumen['pesoano'], 2)


    return filaResumen

# ahora se aplica la funcion de transformacion a cada uno de los dataframes
pd.options.display.max_rows = None
for year in range(firstECHyear, lastECHyear+1):
    datosAgregados = ECHMontevideoDfDictTodosAños[str(year)].groupby(
        ['secc', 'segm']).apply(aggregateECH, args=year)
    datosAgregados.to_csv("ECH/{:}-Agregados.csv".format(year))
    print(f'{year} procesado')



In [ ]:
from IPython.display import display, HTML

display(
    HTML("<div style='height: 500px'>" + datosAgregados.style.to_html() + "</div>")
)